In [24]:
# https://www.ibm.com/think/topics/transformer-model
# https://blogs.nvidia.com/blog/what-is-a-transformer-model/

import trafilatura
html = trafilatura.fetch_url("https://www.ibm.com/think/topics/transformer-model")
text = trafilatura.extract(html)

In [ ]:
def google_parser(search_query, output_file="results.txt", num_results=10, delay=5):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/",
    }

    collected_links = []
    try:
        url = f"https://www.google.com/search?q={urllib.parse.quote_plus(search_query)}&num={num_results}"
        response = requests.get(url, headers=headers)
        print("Status Code:", response.status_code)  # Отладка

        soup = BeautifulSoup(response.text, 'html.parser')
        # Попробуйте разные селекторы:
        for a in soup.select('div.tF2Cxc a, div.yuRUbf a, div.g a'):
            link = a['href']
            if link.startswith("/url?"):
                link = urllib.parse.parse_qs(urllib.parse.urlparse(link).query)['q'][0]
            collected_links.append(link)

        # Сохранение в файл
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("\n".join(collected_links[:num_results]))

    except Exception as e:
        print("Error:", e)

Найдено 0 ссылок. Результаты сохранены в results.txt


In [25]:
text

'The transformer model is a type of neural network architecture that excels at processing sequential data, most prominently associated with large language models (LLMs). Transformer models have also achieved elite performance in other fields of artificial intelligence (AI), such as computer vision, speech recognition and time series forecasting.\nThe transformer architecture was first described in the seminal 2017 paper "Attention is All You Need" by Vaswani and others, which is now considered a watershed moment in deep learning.\nOriginally introduced as an evolution of the recurrent neural network (RNN)-based sequence-to-sequence models used for machine translation, transformer-based models have since attained cutting-edge advancements across nearly every machine learning (ML) discipline.\nDespite their versatility, transformer models are still most commonly discussed in the context of natural language processing (NLP) use cases, such as chatbots, text generation, summarization, ques

In [10]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import time

def google_parser(search_query, output_file="results.txt", num_results=10, delay=5):
    """
    Парсит английские результаты Google с обновленными селекторами
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }

    collected_links = []
    
    try:
        # Кодируем запрос для URL
        encoded_query = urllib.parse.quote_plus(search_query)
        url = f"https://www.google.com/search?q={encoded_query}&num={num_results}&gl=us&hl=en"
        
        response = requests.get(url, headers=headers, timeout=10)
        
        # Проверка на капчу
        if "CAPTCHA" in response.text or "demanded" in response.text:
            print("Google требует CAPTCHA! Используйте прокси или ручной вход.")
            return

        # Сохраняем сырой HTML для отладки
        with open("google_dump.html", "w", encoding="utf-8") as f:
            f.write(response.text)

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Актуальные селекторы на июль 2024
        for result in soup.select('div.g, div.tF2Cxc, div.yuRUbf'):
            link = result.find('a', href=True)
            if link:
                href = link['href']
                # Обрабатываем Google-редиректы
                if href.startswith('/url?'):
                    params = urllib.parse.parse_qs(urllib.parse.urlparse(href).query)
                    href = params.get('q', [href])[0]
                collected_links.append(href)

        # Сохраняем результаты
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("\n".join(collected_links[:num_results]))

        print(f"Found {len(collected_links)} links. Saved to {output_file}")

    except Exception as e:
        print(f"Error: {str(e)}")

# Пример использования (английский запрос)
google_parser("machine learning transformers research papers", num_results=15)

Found 0 links. Saved to results.txt


In [ ]:
import scrapy
from scrapy.linkextractors import LinkExtractor
from urllib.parse import urlparse

class MySpider(scrapy.Spider):
    name = 'link_spider'
    start_urls = ['https://www.google.com']  # Укажите стартовый URL

    def parse(self, response):
        # Инициализируем LinkExtractor с фильтрами
        link_extractor = LinkExtractor(
            allow_domains=[urlparse(response.url).hostname],  # Только текущий домен
            deny=[],  # При необходимости добавьте исключения
            canonicalize=True
        )

        # Собираем данные
        data = {
            'url': [],
            'link_text': [],
            'source_page': []
        }

        # Извлекаем все ссылки с их текстом
        for link in link_extractor.extract_links(response):
            data['url'].append(link.url)
            data['link_text'].append(link.text.strip())
            data['source_page'].append(response.url)

        # Дополнительно: парсим текст со страницы
        text_content = ' '.join(response.xpath('//body//text()').getall()).strip()
        text_content = ' '.join(text_content.split())  # Удаляем лишние пробелы

        # Для экспорта в DataFrame (но лучше использовать Scrapy Items)
        yield {
            'page_url': response.url,
            'page_content': text_content,
            'links': data
        }

        # Рекурсивный переход по ссылкам (опционально)
        for link in link_extractor.extract_links(response):
            yield response.follow(link.url, callback=self.parse)



import scrapy
from scrapy.crawler import CrawlerProcess


process = CrawlerProcess(settings={
    'FEED_FORMAT': 'json',
    'FEED_URI': 'output.json'
})
process.crawl(MySpider)
process.start()

<generator object MySpider.parse at 0x0000014EDEFF3040>

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
import time

def google_selenium(query, num=10):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")
    
    driver = webdriver.Chrome(options=options)
    
    # Маскируем Selenium
    stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True
    )
    
    driver.get(f"https://www.google.com/search?q={query}&num={num}")
    time.sleep(3)
    
    # Извлекаем ссылки
    links = [a.get_attribute('href') for a in driver.find_elements("css selector", "a")]
    driver.quit()
    
    # Фильтруем результаты
    return [link for link in links if 'url?q=' in link][:num]

# Пример использования
print(google_selenium("Transformers in machine learning"))

[]


In [ ]:
def get_yandex_results(query):
    chrome_options = Options()
    chrome_options.add_argument("--headless")  
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")  
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    chrome_options.add_argument("--lang=ru-RU")
    
    driver_path = r"D:\ethd\ml\chromedriver-win64\chromedriver.exe"

    service = Service(driver_path)

    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    try:
        url = f"https://yandex.ru/search/?text={query.replace(' ', '+')}"
        driver.get(url)
        
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "[data-cid]"))
        )
        
        last_height = driver.execute_script("return document.body.scrollHeight")
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        
        links = []
        results = driver.find_elements(By.CSS_SELECTOR, "li.serp-item a.Link_theme_outer")
        
        excluded_keywords = {"course", "edu", "lesson", "school", 'курс', "урок"}
        
        for link in results:
            href = link.get_attribute("href")
            if href and not href.startswith("https://yandex.ru"):
                if href and not any(keyword in href.lower() for keyword in excluded_keywords) and href not in links:
                    links.append(href)
                
        return list(set(links))[:20]
        
    except Exception as e:
        print(f"Ошибка: {e}")
        return []
    finally:
        driver.quit()  

In [ ]:
# https://www.ibm.com/think/topics/transformer-model
# https://blogs.nvidia.com/blog/what-is-a-transformer-model/

import trafilatura
html = trafilatura.fetch_url("https://www.ibm.com/think/topics/transformer-model")
text = trafilatura.extract(html)

In [4]:
from googlesearch import search

def get_google_results(query, num_results):

    results = list(
        search(
            query, 
            num_results=num_results,
            lang="en",
            advanced=False
            ))
    return results

links = get_google_results('Transformers in machine learning', num_results=10)

result_path = r'D:\ethd\ml\Neuro-research\utils\test_links.txt'
with open(result_path, "w", encoding="utf-8") as file:
    for link in links:
        file.write(link + "\n")
    

In [ ]:
from threading import Thread

def get_page_content(link, timeout=5):
    result = None
    def task():
        nonlocal result
        try:
            html = trafilatura.fetch_url(link)
            result = trafilatura.extract(html)
        except Exception:
            pass
            
    thread = Thread(target=task)
    thread.start()
    thread.join(timeout)
    return result

if __name__ == '__main__':
    links = []
    result_path = r'D:\ethd\ml\Neuro-research\utils\test_links.txt'
    
    # Читаем ссылки
    with open(result_path, "r", encoding="utf-8") as file:
        for line in file:
            stripped_line = line.strip()
            if stripped_line:
                links.append(stripped_line)

    # Сохраняем результаты
    result_path = r'D:\ethd\ml\Neuro-research\utils\test_content.txt'
    with open(result_path, "a", encoding="utf-8") as file:
        for link in links:
            content = get_page_content(link)
            # Добавляем проверку на None
            file.write('-'*80)
            file.write('\n')
            file.write(link)
            file.write('\n')
            file.write('-'*80)
            file.write(f"{content or ''}\n")

ERROR:trafilatura.utils:parsed tree length: 1, wrong data type or not valid HTML
ERROR:trafilatura.core:empty HTML tree: None
ERROR:trafilatura.downloads:download error: https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04 HTTPSConnectionPool(host='medium.com', port=443): Max retries exceeded with url: /inside-machine-learning/what-is-a-transformer-d07dd1fbec04 (Caused by ProtocolError('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)))
ERROR:trafilatura.downloads:download error: https://www.datacamp.com/tutorial/how-transformers-work HTTPSConnectionPool(host='www.datacamp.com', port=443): Max retries exceeded with url: /tutorial/how-transformers-work (Caused by ProtocolError('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None)))
ERROR:trafilatura.downloads:download error: https://aws.amazon.com

In [7]:
import trafilatura
import multiprocessing
from queue import Empty

# Выносим worker в глобальную область видимости
def worker(link, queue):
    try:
        html = trafilatura.fetch_url(link)
        text = trafilatura.extract(html)
        queue.put(text or None)
    except Exception:
        queue.put(None)

def get_page_content(link, timeout=10):
    queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=worker, args=(link, queue))
    process.start()
    process.join(timeout=timeout)

    if process.is_alive():
        process.terminate()
        process.join()
        return None
    else:
        try:
            return queue.get_nowait()
        except Empty:
            return None

# Добавляем защиту точки входа
if __name__ == '__main__':
    links = []
    result_path = r'D:\ethd\ml\Neuro-research\utils\test_links.txt'
    
    # Читаем ссылки
    with open(result_path, "r", encoding="utf-8") as file:
        for line in file:
            stripped_line = line.strip()
            if stripped_line:
                links.append(stripped_line)

    # Сохраняем результаты
    result_path = r'D:\ethd\ml\Neuro-research\utils\test_content.txt'
    with open(result_path, "a", encoding="utf-8") as file:
        for link in links:
            content = get_page_content(link)
            # Добавляем проверку на None
            file.write(f"{content or ''}\n")

In [ ]:
data = get_page_content('https://www.ibm.com/think/topics/transformer-model')
data

'The transformer model is a type of neural network architecture that excels at processing sequential data, most prominently associated with large language models (LLMs). Transformer models have also achieved elite performance in other fields of artificial intelligence (AI), such as computer vision, speech recognition and time series forecasting.\nThe transformer architecture was first described in the seminal 2017 paper "Attention is All You Need" by Vaswani and others, which is now considered a watershed moment in deep learning.\nOriginally introduced as an evolution of the recurrent neural network (RNN)-based sequence-to-sequence models used for machine translation, transformer-based models have since attained cutting-edge advancements across nearly every machine learning (ML) discipline.\nDespite their versatility, transformer models are still most commonly discussed in the context of natural language processing (NLP) use cases, such as chatbots, text generation, summarization, ques

In [ ]:
from googlesearch import search

query = "Transformers in machine learning"
num_results = 10

results = list(search(
    query, 
    num_results=num_results,
    lang="en",
    advanced=False
))

print("Найдено ссылок:", len(results))
for url in results:
    print(url)

Найдено ссылок: 10
https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)
https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04
https://blogs.nvidia.com/blog/what-is-a-transformer-model/
https://www.ibm.com/think/topics/transformer-model
https://www.datacamp.com/tutorial/how-transformers-work
https://www.reddit.com/r/explainlikeimfive/comments/16y59y3/eli5_what_are_transformers_in_ml/
https://aws.amazon.com/what-is/transformers-in-artificial-intelligence/
https://www.geeksforgeeks.org/getting-started-with-transformers/
https://web.stanford.edu/~jurafsky/slp3/9.pdf
https://www.youtube.com/watch?v=ZXiruGOCn9s


In [ ]:

import scrapy
from scrapy.crawler import CrawlerProcess


process = CrawlerProcess(settings={
    'FEED_FORMAT': 'json',
    'FEED_URI': 'output.json'
})
process.crawl(MySpider)
process.start()

In [28]:
def responsing(text: str, max_results: int) -> list:
    all_texts = []
    url = "http://localhost:5000/translate"
    data = {
        "q": text,
        "source": "ru",
        "target": "en"
    }
    response = requests.post(url, json=data)
    result = response.json()
    text = result['translatedText']
    print(text)
    arxiv_results = search_arxiv(text, max_results=5)
    print("arXiv results:")
    for res in arxiv_results:
        all_texts.append(extract_text_from_pdf_url(res['pdf_url']))

    return all_texts

print(responsing('трансформеры', 2)[0])

transformer
arXiv results:


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

5
2
0
2

r
a

M
8
2

]
I

N
.
s
c
[

1
v
3
6
6
2
2
.
3
0
5
2
:
v
i
X
r
a

NetSSM: Multi-Flow and State-Aware Network Trace
Generation using State-Space Models

ANDREW CHU, University of Chicago
XI JIANG, University of Chicago
SHINAN LIU, University of Chicago
ARJUN BHAGOJI, IIT Bombay
FRANCESCO BRONZINO, École Normale Supérieure de Lyon
PAUL SCHMITT, California Polytechnic State University, San Luis Obispo
NICK FEAMSTER, University of Chicago

Access to raw network traffic data is essential for many computer networking tasks, from traffic modeling to
performance evaluation. Unfortunately, this data is scarce due to high collection costs and governance rules.
Previous efforts explore this challenge by generating synthetic network data, but fail to reliably handle multi-
flow sessions, struggle to reason about stateful communication in moderate to long-duration network sessions,
and lack robust evaluations tied to real-world utility. We propose a new method based on state-space models
ca

In [4]:
arxiv_results = search_arxiv("transformers in NLP", max_results=5)
print("arXiv results:")
for res in arxiv_results:
    print(f"- {res['title']} ({res['published'][:1]})")

arXiv results:
- Q-Insight: Understanding Image Quality via Visual Reinforcement Learning (2)
- DSO: Aligning 3D Generators with Simulation Feedback for Physical
  Soundness (2)
- Self-Evolving Multi-Agent Simulations for Realistic Clinical
  Interactions (2)
- TranSplat: Lighting-Consistent Cross-Scene Object Transfer with 3D
  Gaussian Splatting (2)
- Think Before Recommend: Unleashing the Latent Reasoning Power for
  Sequential Recommendation (2)


In [5]:
print(arxiv_results[0]['pdf_url'])

http://arxiv.org/pdf/2503.22679v1


In [1]:
import wikipedia
wikipedia.set_lang("en")
content = wikipedia.summary("Transformer (deep learning architecture)")

In [2]:
content

'The transformer is a deep learning architecture that was developed by researchers at Google and is based on the multi-head attention mechanism, which was proposed in the 2017 paper "Attention Is All You Need". Text is converted to numerical representations called tokens, and each token is converted into a vector via lookup from a word embedding table. At each layer, each token is then contextualized within the scope of the context window with other (unmasked) tokens via a parallel multi-head attention mechanism, allowing the signal for key tokens to be amplified and less important tokens to be diminished.\nTransformers have the advantage of having no recurrent units, therefore requiring less training time than earlier recurrent neural architectures (RNNs) such as long short-term memory (LSTM). Later variations have been widely adopted for training large language models (LLM) on large (language) datasets.\n\nTransformers were first developed as an improvement over previous architecture

In [6]:
from collections import defaultdict
from urllib.parse import quote, urlparse, parse_qs
from httpx import Client
from parsel import Selector
import time
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 1. Создаем HTTP клиент с заголовками, имитирующими браузер
client = Client(
    headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9,ru;q=0.8",
    },
    follow_redirects=True,
    http2=True,
    timeout=30
)

def clean_url(url: str) -> str:
    """Очищает URL от трекинговых параметров и редиректов Google"""
    parsed = urlparse(url)
    
    # Для Google-редиректов
    if parsed.netloc.endswith("google.com") and parsed.path == "/url":
        return parse_qs(parsed.query).get("q", [""])[0]
    
    return url

def parse_search_results(selector: Selector):
    """Парсит результаты поиска с Google страницы"""
    results = []
    
    # Используем актуальные селекторы для Google Search
    for box in selector.xpath("//div[@id='search']//div[@class='tF2Cxc']"):
        # Извлечение заголовка
        title_elem = box.xpath(".//h3/text()").get()
        # Извлечение ссылки
        link_elem = box.xpath(".//a/@href").get()
        # Извлечение описания
        snippet_elem = box.xpath(".//div[@data-sncf]//text()").getall()
        
        if not title_elem or not link_elem:
            continue
        
        title = title_elem.strip()
        url = clean_url(link_elem)
        snippet = " ".join([s.strip() for s in snippet_elem])[:200]  # Ограничиваем длину сниппета

        if not url or "google.com" in url:
            continue

        results.append({
            "title": title,
            "url": url,
            "snippet": snippet
        })
    
    return results

def scrape_search(query: str, page=1):
    """Парсит результаты поиска для заданного запроса"""
    # Формируем URL
    url = f"https://www.google.com/search?hl=en&q={quote(query)}"
    if page > 1:
        url += f"&start={10 * (page - 1)}"
    
    logger.info(f"Парсинг {query=} {page=}")
    
    try:
        response = client.get(url)
        response.raise_for_status()
        
        # Проверяем, не требует ли Google капчу
        selector = Selector(response.text)
        
        # Проверка на наличие капчи
        if selector.xpath("//div[contains(text(), 'Our systems have detected unusual traffic')]").get():
            logger.warning("Обнаружена капча от Google. Попробуйте позже или используйте Google Programmable Search Engine")
            return {"query": query, "page": page, "results": []}
        
        # Парсим результаты
        search_results = parse_search_results(selector)
        logger.info(f"Найдено {len(search_results)} результатов на странице {page}")
        
        return {
            "query": query,
            "page": page,
            "results": search_results
        }
    
    except Exception as e:
        logger.error(f"Ошибка при парсинге {query} (страница {page}): {e}")
        return {"query": query, "page": page, "results": []}

# Пример использования
if __name__ == "__main__":
    query = "Transformers in machine learning"
    for page in [1, 2, 3]:
        try:
            results = scrape_search(query, page=page)
            
            print(f"\nРезультаты поиска для '{query}' (страница {page}):")
            if results["results"]:
                for i, result in enumerate(results["results"], 1):
                    print(f"{i}. {result['title']}")
                    print(f"   {result['url']}")
                    print(f"   {result['snippet']}\n")
            else:
                print("   Нет результатов или страница заблокирована Google.\n")
            
            time.sleep(3)  # Добавляем паузу между страницами
        
        except Exception as e:
            logger.error(f"Ошибка при обработке страницы {page}: {e}")
            continue

INFO:__main__:Парсинг query='Transformers in machine learning' page=1
INFO:httpx:HTTP Request: GET https://www.google.com/search?hl=en&q=Transformers%20in%20machine%20learning "HTTP/2 200 OK"
INFO:__main__:Найдено 0 результатов на странице 1



Результаты поиска для 'Transformers in machine learning' (страница 1):
   Нет результатов или страница заблокирована Google.



INFO:__main__:Парсинг query='Transformers in machine learning' page=2
INFO:httpx:HTTP Request: GET https://www.google.com/search?hl=en&q=Transformers%20in%20machine%20learning&start=10 "HTTP/2 200 OK"
INFO:__main__:Найдено 0 результатов на странице 2



Результаты поиска для 'Transformers in machine learning' (страница 2):
   Нет результатов или страница заблокирована Google.



INFO:__main__:Парсинг query='Transformers in machine learning' page=3
INFO:httpx:HTTP Request: GET https://www.google.com/search?hl=en&q=Transformers%20in%20machine%20learning&start=20 "HTTP/2 200 OK"
INFO:__main__:Найдено 0 результатов на странице 3



Результаты поиска для 'Transformers in machine learning' (страница 3):
   Нет результатов или страница заблокирована Google.



In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import torch
import re


model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def translate(text: str) -> str:
    """Переводит текст с русского на английский"""
    tokenized_text = tokenizer.prepare_seq2seq_batch([text], return_tensors='pt').to(device)
    translated = model.generate(**tokenized_text)
    result = tokenizer.decode(translated[0], skip_special_tokens=True)
    return result

def translate_file(file_path: str):
    """Читает файл, переводит текст и сохраняет результат обратно"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            input_text = file.read()

        print("Файл успешно прочитан.")

        translated_text = translate(input_text)

        print("Перевод завершён.")

        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(translated_text)

        print(f"Перевод сохранён в файл: {file_path}")

    except Exception as e:
        print(f"Произошла ошибка: {e}")

def clean_text(text):
    # Удаление HTML-тегов
    text = re.sub(r"<[^>]+>", "", text)
    
    # Удаление URL
    text = re.sub(r"https?://\S+|www\.\S+", "", text)
    
    # Удаление повторяющихся строк
    lines = text.split("\n")
    seen = set()
    cleaned_lines = []
    for line in lines:
        line = line.strip()
        if line and line not in seen:
            seen.add(line)
            cleaned_lines.append(line)
    text = "\n".join(cleaned_lines)
    
    # Удаление технических строк
    text = re.sub(r"(?i)(?:etext|from|q)=\S+", "", text)

    return text

def translate_file_in_parts(file_path: str):
    with open(file_path, 'r', encoding='utf-8') as file:
        input_text = file.read()
    
    print(f"Исходный текст: {len(input_text)} символов")
    
    cleaned_text = clean_text(input_text)
    print(f"После очистки: {len(cleaned_text)} символов")
    
    translated_text = translate_in_chunks(cleaned_text)
    print(f"После перевода: {len(translated_text)} символов")
    
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(translated_text)
    
    print("Перевод завершён.")

def translate_in_chunks(text, chunk_size=1000):
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    translated = []

    for i, chunk in enumerate(chunks):
        print(f"Перевод части {i+1}/{len(chunks)}")
        try:
            result = translate(chunk)
            if result:
                translated.append(result)
            else:
                print(f"Часть {i+1} не переведена")
        except Exception as e:
            print(f"Ошибка при переводе части {i+1}: {e}")

    return "\n".join(translated)


file_path = r"D:\ethd\печать\fceb0dad-0cf7-40b7-9c86-2da0ea118b91_page.txt"
translate_file_in_parts(file_path)

d:\ethd\conda\envs\neuro-research-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\ethd\conda\envs\neuro-research-env\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Исходный текст: 36867 символов
После очистки: 36705 символов
Перевод части 1/37


d:\ethd\conda\envs\neuro-research-env\Lib\site-packages\transformers\tokenization_utils_base.py:4106: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Перевод части 2/37
Перевод части 3/37
Перевод части 4/37
Перевод части 5/37
Перевод части 6/37
Перевод части 7/37
Перевод части 8/37
Перевод части 9/37
Перевод части 10/37
Перевод части 11/37
Перевод части 12/37
Перевод части 13/37
Перевод части 14/37
Перевод части 15/37
Перевод части 16/37
Перевод части 17/37
Перевод части 18/37
Перевод части 19/37
Перевод части 20/37
Перевод части 21/37
Перевод части 22/37
Перевод части 23/37
Перевод части 24/37
Перевод части 25/37
Перевод части 26/37
Перевод части 27/37
Перевод части 28/37
Перевод части 29/37
Перевод части 30/37
Перевод части 31/37
Перевод части 32/37
Перевод части 33/37
Перевод части 34/37
Перевод части 35/37
Перевод части 36/37
Перевод части 37/37
После перевода: 25928 символов
Перевод завершён.
